## Supplemental analyses for the manuscript "Cultural Individualism/Collectivism Moderates the Effects of Climate Action Interventions"

In [1]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
import bff
import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

Autosaving every 30 seconds


### Import data and run data preprocessing

In [2]:
#import main df
df = pd.read_excel("data63.xlsx")

In [3]:
df['ide'] = (df['Politics2_1'] + df['Politics2_9'])/2

In [4]:
#load predictor dataframe
predictordata = pd.read_excel("PredictorData.xlsx")

#merge main dataframe with predictor data & wvs survey data
df = df.merge(predictordata, on='Country', how='left')

In [5]:
df = df.rename(columns={"IndividualismHI": "Ind_score"})
#create variable split around 50
df['Ind_ms'] = df['Ind_score'].apply(lambda x: 1 if x < 50 else 2)

#global split
df_col = df.loc[df['Ind_ms']==1] #dataframe for collectivist countries
df_ind = df.loc[df['Ind_ms']==2] #dataframe for individualist countries
df = df.dropna(subset=['Ind_ms'])

In [12]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython

%R library(lme4)
%R library(lmerTest)
%R library(EMAtools)
%R library(effectsize)
%R library(ordinal)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In addition: Warning message:
package ‘effectsize’ was built under R version 4.3.3 


array(['effectsize', 'EMAtools', 'ordinal', 'lmerTest', 'lme4', 'Matrix',
       'tools', 'stats', 'graphics', 'grDevices', 'utils', 'datasets',
       'methods', 'base'], dtype='<U10')

# 1. Robustness checks of individualism’s impact on outcomes adjusting for additional covariates

In [7]:
#create control condition dataframe
dfc = df[df['condName'] == "Control"]
dfc['condName'] = dfc['condName'].replace(['Control'], 'aControl')

## Belief

In [8]:
dfc_belief = pd.melt(
    dfc.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide',  
                'Belief1', 'Belief2', 'Belief3', 'Belief4', "RelMobility", "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"]],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide',  
             "RelMobility",  "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"],
    var_name="Item",
    value_name="Belief"
)

In [9]:
dfc_belief = dfc_belief.dropna(subset=["Belief"]) #deal with missing data for key outcome variable
%Rpush dfc_belief

### Table S1. individual-level covariates and GDP

In [13]:
%%R

# Remove rows with missing gender data and keep only rows where Gender is 1 or 2
dfc_belief <- dfc_belief[!is.na(dfc_belief$Gender) & dfc_belief$Gender %in% c(1, 2), ]

# Remove rows with missing data for the other variables included in the model
dfc_beliefC <- dfc_belief[complete.cases(dfc_belief[, c("Ind_score", "Income", "Edu", "Age", "ide", "GDP", "Belief", "Country", "ResponseId", "Item")]), ]

# Standardize variables to avoid convergence issues
dfc_beliefC$Income <- scale(dfc_beliefC$Income)
dfc_beliefC$Edu <- scale(dfc_beliefC$Edu)
dfc_beliefC$Age <- scale(dfc_beliefC$Age)
dfc_beliefC$ide <- scale(dfc_beliefC$ide)
dfc_beliefC$GDP <- scale(dfc_beliefC$GDP)

# Run the model
M1 <- lmer(Belief ~ Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_beliefC)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_1_indcovariates.csv") #export coefficients to csv
# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = dfc_belief, type = "lme4")
print(cohens_d)


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_beliefC

REML criterion at convergence: 125864.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2947 -0.2868  0.0897  0.3190  5.9598 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 500.2930 22.3672 
 Country    (Intercept)  18.9569  4.3539 
 Item       (Intercept)   0.2925  0.5408 
 Residual               147.0445 12.1262 
Number of obs: 14802, groups:  ResponseId, 3703; Country, 59; Item, 4

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)          82.62131    1.95277   67.86377  42.310  < 2e-16 ***
Ind_score            -0.11802    0.03194   58.50240  -3.695 0.000486 ***
Income                0.96253    0.43824 2372.19222   2.196 0.028165 *  
Edu       

In addition: Warning messages:
1: Some predictor variables are on very different scales: consider rescaling 
2: Some predictor variables are on very different scales: consider rescaling 


### Table S4. parochialism and relational mobility

In [14]:
%%R

#standardize variables
dfc_belief$Parochialism <- scale(dfc_belief$Parochialism)
dfc_belief$RelMobility <- scale(dfc_belief$RelMobility)

M1 <- lmer(Belief ~ Ind_score + Parochialism + RelMobility + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_1_relmob_paroc.csv") #export coefficients to csv

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = dfc_belief, type = "lme4")
print(cohens_d)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ Ind_score + Parochialism + RelMobility + (1 | Country) +  
    (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 78136.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.1039 -0.2950  0.0788  0.3127  6.0623 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 619.4269 24.8883 
 Country    (Intercept)  20.8773  4.5692 
 Item       (Intercept)   0.4927  0.7019 
 Residual               142.9889 11.9578 
Number of obs: 9159, groups:  ResponseId, 2293; Country, 20; Item, 4

Fixed effects:
             Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)  90.34599    3.48444 21.79529  25.928  < 2e-16 ***
Ind_score    -0.19498    0.05357 18.09547  -3.640  0.00186 ** 
Parochialism  1.18089    1.25204 16.44386   0.943  0.35925    
RelMobility   0.75109    1.15944 16.15535   0.648  0.52621    
---
Signif. co

## Policy Support

In [15]:
dfc_policy = pd.melt(
    dfc.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide', 
                'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9', "RelMobility",
                "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"]],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide', "RelMobility",
             "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"],
    var_name="Item",
    value_name="Policy"
)
#deal with missing data
dfc_policy = dfc_policy.dropna(subset=["Policy"])
%Rpush dfc_policy

### Table S2. individual-level covariates and GDP

In [16]:
%%R

# Remove rows with missing gender data and keep only rows where Gender is 1 or 2
dfc_policy <- dfc_policy[!is.na(dfc_policy$Gender) & dfc_policy$Gender %in% c(1, 2), ]

# Remove rows with missing data for the other variables included in the model
dfc_policyC <- dfc_policy[complete.cases(dfc_policy[, c("Ind_score", "Income", "Edu", "Age", "ide", "GDP", "Policy", "Country", "ResponseId", "Item")]), ]

# Standardize variables to avoid convergence issues
dfc_policyC$Income <- scale(dfc_policyC$Income)
dfc_policyC$Edu <- scale(dfc_policyC$Edu)
dfc_policyC$Age <- scale(dfc_policyC$Age)
dfc_policyC$ide <- scale(dfc_policyC$ide)
dfc_policyC$GDP <- scale(dfc_policyC$GDP)

# Run the model
M1 <- lmer(Policy ~ Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policyC)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_2_indcovariates.csv") #export coefficients to csv

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = dfc_policyC, type = "lme4")
print(cohens_d)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policyC

REML criterion at convergence: 295198.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6039 -0.5258  0.0629  0.6270  4.2268 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 314.94   17.747  
 Country    (Intercept)  13.54    3.679  
 Item       (Intercept) 146.32   12.096  
 Residual               406.46   20.161  
Number of obs: 32503, groups:  ResponseId, 3695; Country, 59; Item, 9

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)          73.76897    4.34459   10.73322  16.979 4.25e-09 ***
Ind_score            -0.09732    0.02673   58.64447  -3.641 0.000575 ***
Income                0.44178    0.35986 2416.96312   1.228 0.219697    
Edu       

### Table S5. parochialism and relational mobility

In [17]:
%%R

#standardize variables
dfc_policy$Parochialism <- scale(dfc_policy$Parochialism)
dfc_policy$RelMobility <- scale(dfc_policy$RelMobility)

M1 <- lmer(Policy ~ Ind_score + Parochialism + RelMobility + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_2_relmob_paroc.csv") #export coefficients to csv

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = dfc_policy, type = "lme4")
print(cohens_d)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ Ind_score + Parochialism + RelMobility + (1 | Country) +  
    (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 180641

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6307 -0.5420  0.0470  0.6167  4.3826 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 373.66   19.330  
 Country    (Intercept)  10.58    3.253  
 Item       (Intercept) 157.79   12.561  
 Residual               393.70   19.842  
Number of obs: 19914, groups:  ResponseId, 2272; Country, 20; Item, 9

Fixed effects:
             Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)  73.96579    4.92498 14.46442  15.018 3.19e-10 ***
Ind_score    -0.08900    0.03981 17.19621  -2.236   0.0389 *  
Parochialism  1.77942    0.92647 15.22399   1.921   0.0737 .  
RelMobility   0.58154    0.85849 15.22211   0.677   0.5083    
---
Signif. co

## Sharing

In [18]:
dfc_share = dfc.dropna(subset=["SHAREcc"]) #deal with missing data
%Rpush dfc_share

### Table S3. individual-level covariates and GDP

In [19]:
%%R

# Remove rows with missing gender data and keep only rows where Gender is 1 or 2
dfc_share <- dfc_share[!is.na(dfc_share$Gender) & dfc_share$Gender %in% c(1, 2), ]

# Remove rows with missing data for the other variables included in the model
dfc_shareC <- dfc_share[complete.cases(dfc_share[, c("Ind_score", "Income", "Edu", "Age", "ide", "GDP", "SHAREcc", "Country", "ResponseId")]), ]

# Standardize variables to avoid convergence issues
dfc_shareC$Income <- scale(dfc_shareC$Income)
dfc_shareC$Edu <- scale(dfc_shareC$Edu)
dfc_shareC$Age <- scale(dfc_shareC$Age)
dfc_shareC$ide <- scale(dfc_shareC$ide)
dfc_shareC$GDP <- scale(dfc_shareC$GDP)

M1 <- glmer(SHAREcc ~ Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country), family = binomial, data = dfc_shareC)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_3_indcovariates.csv") #export coefficients to csv

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = dfc_shareC, type = "lme4")
print(cohens_d)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country)
   Data: dfc_shareC

     AIC      BIC   logLik deviance df.resid 
  3562.0   3615.7  -1772.0   3544.0     2882 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1753 -0.8085  0.3798  0.7672  3.9544 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.8708   0.9332  
Number of obs: 2891, groups:  Country, 59

Fixed effects:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)         1.102066   0.345397   3.191  0.00142 ** 
Ind_score          -0.016355   0.005803  -2.818  0.00482 ** 
Income             -0.013089   0.050009  -0.262  0.79352    
Edu                 0.120057   0.044323   2.709  0.00675 ** 
Age                -0.255634   0.045142  -5.663 1.49e-08 ***
as.factor(Gender)2 -0.274883   0.084770

### Table S6. parochialism and relational mobility

In [21]:
%%R

#standardize variables
dfc_share$Parochialism <- scale(dfc_share$Parochialism)
dfc_share$RelMobility <- scale(dfc_share$RelMobility)

M1 <- glmer(SHAREcc ~ Ind_score + Parochialism + RelMobility + (1 | Country), family = binomial, data = dfc_share)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_3_relmob_paroc.csv") #export coefficients to csv

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = dfc_shareC, type = "lme4")
print(cohens_d)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ Ind_score + Parochialism + RelMobility + (1 | Country)
   Data: dfc_share

     AIC      BIC   logLik deviance df.resid 
  2185.3   2212.5  -1087.6   2175.3     1702 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.1244 -0.9374 -0.4349  0.9319  2.5059 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.5546   0.7447  
Number of obs: 1707, groups:  Country, 20

Fixed effects:
              Estimate Std. Error z value Pr(>|z|)   
(Intercept)   1.047240   0.496264   2.110   0.0348 * 
Ind_score    -0.020185   0.007804  -2.586   0.0097 **
Parochialism -0.022826   0.186226  -0.123   0.9024   
RelMobility   0.397350   0.174146   2.282   0.0225 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) Ind_sc Prchls
Ind_score   -0.919              
Pa

## Tree Planting

In [23]:
dfc_WEPT = dfc.dropna(subset=["WEPTcc"]) # deal with missing data
%Rpush dfc_WEPT

### parochialism and relational mobility

In [24]:
%%R

M1 <- clmm(as.factor(WEPTcc) ~ Ind_score + Parochialism + RelMobility + (1 | Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
print(summary(M1))

# Extract fixed effects coefficients directly
model_coef <- coef(M1)

# Calculate odds ratios for fixed effects
odds_ratios <- exp(model_coef)

print(odds_ratios)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ Ind_score + Parochialism + RelMobility +  
    (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC     niter    max.grad cond.H 
 logit equidistant 2330 -3628.65 7269.29 278(843) 4.42e-03 2.0e+05

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.04799  0.2191  
Number of groups:  Country 20 

Coefficients:
              Estimate Std. Error z value Pr(>|z|)  
Ind_score    -0.006336   0.003118  -2.032   0.0421 *
Parochialism -0.043179   0.374943  -0.115   0.9083  
RelMobility   0.158611   0.457867   0.346   0.7290  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Threshold coefficients:
            Estimate Std. Error z value
threshold.1 -1.72922    0.19729  -8.765
spacing      0.17325    0.00613  28.261
(2755 observations deleted due to missingness)
 threshold.1      spacing    Ind_score Parochialism  RelMobility 
   0.1774220

# 2. Separate analyses conducted in individualistic and collectivistic countries
#### (split using continuous individualism score from 0-100, split at 50)

# Belief

In [28]:
df_belief = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP", 'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP"],
    var_name="Item",
    value_name="Belief"
)
df_belief['condName'] = df_belief['condName'].replace(['Control'], 'aControl')
df_belief = df_belief.dropna(subset=["ResponseId", "Belief", "condName"])

In [29]:
#create separate dataframes
df_col_belief = df_belief.loc[df_belief['Ind_ms']==1] #dataframe for collectivist countries
df_ind_belief = df_belief.loc[df_belief['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_belief
%Rpush df_ind_belief

### Table S7. Collectivist Countries

In [30]:
%%R

control = lmerControl(optimizer = "bobyqa") #update optimizer
M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_col_belief, control = control)
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_col_belief.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_col_belief
Control: control

REML criterion at convergence: 819139.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.0604 -0.2144  0.1154  0.2876  5.4822 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 337.7082 18.3768 
 Country    (Intercept)  30.1459  5.4905 
 Item       (Intercept)   0.3443  0.5868 
 Residual               162.2154 12.7364 
Number of obs: 96517, groups:  ResponseId, 24187; Country, 38; Item, 4

Fixed effects:
                          Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)              8.259e+01  1.033e+00 5.521e+01  79.947  < 2e-16 ***
condNameBindingMoral     5.196e-01  6.054e-01 2.407e+04   0.858 0.390777    
condNameCollectAction    1.803e+00  6.006e-01 2.407e+04   3.002 0.002684 ** 
condNameDynamicNorm      5.408e-01

### Table S8. Individualist Countries

In [31]:
%%R

control = lmerControl(optimizer = "bobyqa") #update optimizer
M1 <- lmer(Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_ind_belief, control = control)
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_ind_belief.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_ind_belief
Control: control

REML criterion at convergence: 1175136

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.4546 -0.2923  0.0807  0.3003  6.2571 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 612.6246 24.7513 
 Country    (Intercept)  27.5531  5.2491 
 Item       (Intercept)   0.2903  0.5388 
 Residual               138.2471 11.7579 
Number of obs: 138245, groups:  ResponseId, 34595; Country, 25; Item, 4

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 76.1891     1.1851    35.6056  64.287  < 2e-16 ***
condNameBindingMoral         1.5327     0.6617 34552.2882   2.316  0.02055 *  
condNameCollectAction        2.7016     0.6610 34549.6162   4.087 4.37e-05 ***
condNameDynamicNorm       

# Policy Support

In [32]:
df_policy = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP"],
    var_name="Item",
    value_name="Policy"
)
df_policy['condName'] = df_policy['condName'].replace(['Control'], 'aControl')
df_policy = df_policy.dropna(subset=["ResponseId", "Policy", "condName"])

In [33]:
df_col_policy = df_policy.loc[df_policy['Ind_ms']==1] #dataframe for collectivist countries
df_ind_policy = df_policy.loc[df_policy['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_policy
%Rpush df_ind_policy

### Table S9. Collectivist Countries

In [34]:
%%R
M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_col_policy)
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_col_policy.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_col_policy

REML criterion at convergence: 1901984

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6188 -0.4948  0.1103  0.6516  4.3458 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 220.42   14.847  
 Country    (Intercept)  22.03    4.693  
 Item       (Intercept) 125.51   11.203  
 Residual               449.52   21.202  
Number of obs: 208145, groups:  ResponseId, 24037; Country, 38; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 72.6084     3.8291     8.8516  18.962 1.79e-08 ***
condNameBindingMoral         0.5359     0.5159 23797.4363   1.039   0.2990    
condNameCollectAction        2.4242     0.5116 23787.4037   4.739 2.16e-06 ***
condNameDynamicNorm          1.0544     0.5

### Table S10. Individualist Countries

In [35]:
%%R
M1 <- lmer(Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_ind_policy)
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_ind_policy.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_ind_policy

REML criterion at convergence: 2742990

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.8523 -0.5431  0.0448  0.6275  4.8557 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 365.22   19.111  
 Country    (Intercept)  14.38    3.792  
 Item       (Intercept) 153.56   12.392  
 Residual               413.26   20.329  
Number of obs: 301125, groups:  ResponseId, 34345; Country, 25; Item, 9

Fixed effects:
                           Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)                 66.9462     4.2172     8.6712  15.875 1.04e-07 ***
condNameBindingMoral         0.8471     0.5308 34226.4609   1.596  0.11051    
condNameCollectAction        3.2704     0.5307 34217.4239   6.163 7.23e-10 ***
condNameDynamicNorm          1.1913     0.5

# Sharing

In [36]:
cols_to_check = ["ResponseId", "condName", "SHAREcc"]
df_share = df.dropna(subset=cols_to_check)
df_share['condName'] = df_share['condName'].replace(['Control'], 'aControl')

In [37]:
df_col_share = df_share.loc[df_share['Ind_ms']==1] #dataframe for collectivist countries
df_ind_share = df_share.loc[df_share['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_share
%Rpush df_ind_share

### Table S11. Collectivist Countries

In [38]:
%%R
M1 <- glmer(SHAREcc ~ condName + (1 | Country), family = binomial, data = df_col_share)
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_col_share.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName + (1 | Country)
   Data: df_col_share

     AIC      BIC   logLik deviance df.resid 
 20390.7  20492.9 -10182.4  20364.7    19115 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.3089 -0.6039  0.4191  0.5981  2.5610 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1.206    1.098   
Number of obs: 19128, groups:  Country, 38

Fixed effects:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)               0.42911    0.18717   2.293 0.021866 *  
condNameBindingMoral      0.30532    0.08076   3.780 0.000157 ***
condNameCollectAction     0.60396    0.08251   7.320 2.48e-13 ***
condNameDynamicNorm       0.45772    0.08132   5.628 1.82e-08 ***
condNameFutureSelfCont    0.41964    0.08591   4.885 1.03e-06 ***
condNameLetterFutureGen   0.42238    0.08659   4.878 1.07e-06 ***


### Table S12. Individualist Countries

In [39]:
%%R
M1 <- glmer(SHAREcc ~ condName + (1 | Country), family = binomial, data = df_ind_share)
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_ind_share.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName + (1 | Country)
   Data: df_ind_share

     AIC      BIC   logLik deviance df.resid 
 31354.9  31460.3 -15664.4  31328.9    24515 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9760 -0.8137 -0.4808  0.8616  2.4138 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.5497   0.7414  
Number of obs: 24528, groups:  Country, 25

Fixed effects:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -0.54520    0.15538  -3.509 0.000450 ***
condNameBindingMoral      0.11517    0.06454   1.785 0.074328 .  
condNameCollectAction     0.26636    0.06551   4.066 4.78e-05 ***
condNameDynamicNorm       0.21923    0.06499   3.373 0.000742 ***
condNameFutureSelfCont    0.25089    0.06874   3.650 0.000263 ***
condNameLetterFutureGen   0.37938    0.07024   5.401 6.61e-08 ***


# TREE PLANTING

In [40]:
cols_to_check = ["ResponseId", "condName", "WEPTcc"]
df_WEPT = df.dropna(subset=cols_to_check)
df_WEPT['condName'] = df_WEPT['condName'].replace(['Control'], 'aControl')

In [41]:
df_col_WEPT = df_WEPT.loc[df_WEPT['Ind_ms']==1] #dataframe for collectivist countries
df_ind_WEPT = df_WEPT.loc[df_WEPT['Ind_ms']==2] #dataframe for individualist countries
%Rpush df_col_WEPT
%Rpush df_ind_WEPT

### Table S13. Collectivist Countries

In [42]:
%%R
M1 <- clmm(as.factor(WEPTcc) ~ condName + (1 | Country), data = df_col_WEPT, threshold = "equidistant")
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_col_wept.csv") #export coefficients to csv

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_col_WEPT

 link  threshold   nobs  logLik    AIC      niter      max.grad cond.H 
 logit equidistant 24233 -39844.78 79717.55 1899(7601) 1.61e-01 5.7e+03

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2804   0.5295  
Number of groups:  Country 38 

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral     -0.04640    0.05953  -0.779  0.43573    
condNameCollectAction    -0.10225    0.05885  -1.737  0.08230 .  
condNameDynamicNorm      -0.04160    0.05902  -0.705  0.48091    
condNameFutureSelfCont   -0.17534    0.06152  -2.850  0.00437 ** 
condNameLetterFutureGen  -0.24772    0.06118  -4.049 5.14e-05 ***
condNameNegativeEmotions -0.32770    0.05831  -5.619 1.92e-08 ***
condNamePluralIgnorance  -0.13630    0.05934  -2.297  0.02161 *  
condNamePsychDistance    -0.33337    0.06084

### Table S14. Individualist Countries

In [43]:
%%R
M1 <- clmm(as.factor(WEPTcc) ~ condName + (1 | Country), data = df_ind_WEPT, threshold = "equidistant")
print(summary(M1))
#write.csv(coef(summary(M1)), "coef_ind_wept.csv") #export coefficients to csv

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName + (1 | Country)
data:    df_ind_WEPT

 link  threshold   nobs  logLik    AIC       niter      max.grad cond.H 
 logit equidistant 34647 -56474.87 112977.75 1617(4856) 7.17e-02 8.2e+03

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1401   0.3743  
Number of groups:  Country 25 

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral      0.07717    0.04961   1.556  0.11982    
condNameCollectAction    -0.12802    0.04896  -2.615  0.00893 ** 
condNameDynamicNorm       0.03136    0.04930   0.636  0.52474    
condNameFutureSelfCont   -0.08601    0.05206  -1.652  0.09851 .  
condNameLetterFutureGen  -0.21412    0.05257  -4.073 4.64e-05 ***
condNameNegativeEmotions -0.26765    0.04855  -5.512 3.54e-08 ***
condNamePluralIgnorance  -0.02134    0.04897  -0.436  0.66297    
condNamePsychDistance    -0.23899    0.049

# 3. Analyses of interventions’ impact on outcomes using linear mixed effects models including by-country random slopes.

In [53]:
#import main df
df = pd.read_excel("data63.xlsx")

#load predictor dataframe
predictordata = pd.read_excel("PredictorData.xlsx")

#merge main dataframe with predictor data & wvs survey data
df = df.merge(predictordata, on='Country', how='left')

In [54]:
df = df.rename(columns={"IndividualismHI": "Ind_score"})
#create variable split around 50
df['Ind_ms'] = df['Ind_score'].apply(lambda x: 1 if x < 50 else 2)

#global split
df_col = df.loc[df['Ind_ms']==1] #dataframe for collectivist countries
df_ind = df.loc[df['Ind_ms']==2] #dataframe for individualist countries
df = df.dropna(subset=['Ind_ms'])

## Interventions’ Impact on Beliefs

In [55]:
df = df.dropna(subset=["BELIEFcc", "Country", "condName", "Ind_score", "POLICYcc"])
df['condName'] = df['condName'].replace(['Control'], 'AControl')
df

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,BELIEFcc,POLICYcc,GDP,Ind_score,MinkovKaasa2022,RelMobility,GLOBECol2SocPracInGroup,GLOBECol1SocValInstit,Parochialism,Ind_ms
0,R_1r7Yr7R1xMimIvj,usa,5,CollectAction,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,98.0,77.0,86.0,93.0,71.0,...,88.50,89.000000,25464.475,91.0,33.0,0.182,4.253546,4.16578,0.01147,2
1,R_2ATgV5VxewrTRiI,usa,1,AControl,PolicySupport|BeliefinAnthrCC|Socialmediashari...,49.0,50.0,51.0,51.0,1.0,...,50.25,66.222222,25464.475,91.0,33.0,0.182,4.253546,4.16578,0.01147,2
2,R_3ENlePBlLyYjExr,usa,9,LetterFutureGen,SocialmediasharingPipedText|PolicySupport|Beli...,87.0,82.0,84.0,83.0,84.0,...,84.00,84.222222,25464.475,91.0,33.0,0.182,4.253546,4.16578,0.01147,2
3,R_2c0HQgvYzas6mus,usa,4,SciConsens,PolicySupport|SocialmediasharingPipedText|Beli...,1.0,1.0,0.0,0.0,1.0,...,0.50,1.222222,25464.475,91.0,33.0,0.182,4.253546,4.16578,0.01147,2
4,R_00siryCwgdMus6J,usa,8,PluralIgnorance,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,56.0,59.0,61.0,54.0,79.0,...,57.50,80.222222,25464.475,91.0,33.0,0.182,4.253546,4.16578,0.01147,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59435,R_2TuXye2HP6EO5KZ,vietnam,11,FutureSelfCont,PolicySupport|SocialmediasharingPipedText|Beli...,49.0,100.0,100.0,74.0,70.0,...,80.75,82.500000,406.452,20.0,-78.0,NaN,NaN,NaN,NaN,1
59436,R_3R3p8C0gzLKTcCu,vietnam,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,94.0,98.0,98.0,96.0,92.0,...,96.50,86.444444,406.452,20.0,-78.0,NaN,NaN,NaN,NaN,1
59437,R_OeT1JV3yC1aJ4EF,vietnam,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,100.0,100.0,100.0,100.0,80.0,...,100.00,83.333333,406.452,20.0,-78.0,NaN,NaN,NaN,NaN,1
59438,R_2azVf0KXD3rtly8,vietnam,3,NegativeEmotions,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,79.0,51.0,78.0,51.0,0.0,...,64.75,66.666667,406.452,20.0,-78.0,NaN,NaN,NaN,NaN,1


### Table S15. Belief

In [ ]:
md = smf.mixedlm("BELIEFcc ~ condName*Ind_score", df, groups=df["Country"], re_formula="~condName")
mdf = md.fit()
print(mdf.summary())

/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Che

                                  Mixed Linear Model Regression Results
Model:                            MixedLM                 Dependent Variable:                 BELIEFcc    
No. Observations:                 43350                   Method:                             REML        
No. Groups:                       62                      Scale:                              534.0218    
Min. group size:                  88                      Log-Likelihood:                     -197915.3453
Max. group size:                  6111                    Converged:                          No          
Mean group size:                  699.2                                                                   
----------------------------------------------------------------------------------------------------------
                                                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------

/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


### Table S16. Policy

In [ ]:
md = smf.mixedlm("POLICYcc ~ condName*Ind_score", df, groups=df["Country"], re_formula="~condName")
mdf = md.fit()
print(mdf.summary())

/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(


                                  Mixed Linear Model Regression Results
Model:                            MixedLM                 Dependent Variable:                 POLICYcc    
No. Observations:                 43350                   Method:                             REML        
No. Groups:                       62                      Scale:                              359.2549    
Min. group size:                  88                      Log-Likelihood:                     -189227.4044
Max. group size:                  6111                    Converged:                          Yes         
Mean group size:                  699.2                                                                   
----------------------------------------------------------------------------------------------------------
                                                                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------------

/Users/madalina/anaconda3/lib/python3.11/site-packages/statsmodels/regression/mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


### Table S17. Sharing

In [60]:
#deal with missing data
df_share = df.dropna(subset=["SHAREcc"])

df_share['condName'] = df_share['condName'].replace(['Control'], 'aControl')

# Mean Imputation for specified columns
columns_to_impute = ["Income", "Edu", "Age", "Gender", "GDP"]
for col in columns_to_impute:
    df_share[col] = df_share[col].fillna(df_share[col].mean())
%Rpush df_share

In [ ]:
%%R
# This model adds the random slope of the main effect of condition by country 
# Can finish running but cannot converge
# Please run this code

df_share$Ind_score <- scale(df_share$Ind_score)
#this code rescales the predictor variable to address large eigenvalues

M1 <- glmer(SHAREcc ~ condName*Ind_score + (condName | Country), 
            family = binomial, 
            data = df_share, 
            control = glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 100000)))
#this code changes the optimizer to address model convergence: lme4 uses "bobyqa" by default for GLMMs, but try others like "Nelder_Mead" or "L-BFGS-B".
print(summary(M1))
summary_M1 <- summary(M1)

#rsq <- r2(M1) #marginal and conditional R² values
#print(rsq)

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = df_share, type = "lme4")
print(cohens_d)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName * Ind_score + (condName | Country)
   Data: df_share
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 1e+05))

     AIC      BIC   logLik deviance df.resid 
 51935.5  52821.5 -25865.8  51731.5    43628 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1726 -0.7492  0.3734  0.7814  2.5372 

Random effects:
 Groups  Name                     Variance  Std.Dev. Corr                   
 Country (Intercept)              0.9960169 0.99801                         
         condNameBindingMoral     0.0002781 0.01668   1.00                  
         condNameCollectAction    0.0327918 0.18108  -0.16 -0.16            
         condNameDynamicNorm      0.0200555 0.14162   0.14  0.14  0.42      
         condNameFutureSelfCont   0.0256216 0.16007   0.11  0.11  0.67  0.75
         condNameLetterFutureGen  0.0247193 0.

boundary (singular) fit: see help('isSingular')

Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it

boundary (singular) fit: see help('isSingular')
In addition: Warning messages:
1: In commonArgs(par, fn, control, environment()) :
  maxfun < 10 * length(par)^2 is not recommended.
2: Model failed to converge with 8 negative eigenvalues: -1.4e+03 -2.0e+03 -7.0e+03 -1.1e+04 -2.5e+04 -3.7e+04 -4.2e+04 -1.0e+05 


# 4. Analyses of interventions’ impact on outcomes using continents as random effects to control for geographical cultural interdependence.

In [ ]:
#import continent data
continent_data = pd.read_csv("continent.csv")
continent_data.rename(columns={'country': 'Country'}, inplace=True)
continent_data.rename(columns={'continent': 'Continent'}, inplace=True)
continent_data.drop(columns='unMember', inplace=True)
continent_data['Country'] = continent_data['Country'].str.lower()
continent_data.head()

,Country,Continent
0,india,Asia
1,china,Asia
2,usa,North America
3,indonesia,Asia
4,pakistan,Asia


In [ ]:
# merge with main dataframe
df = df.merge(continent_data, on='Country', how='left')
df.head()

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,PowerDistanceHI,Ind_score,MasculinityHI,UncertaintyAvoidanceHI,LongTermOrientationHI,IndulgenceHI,Notes,MinkovKaasa2022,Ind_ms,Continent
0,R_1DNJsF7DGNuIb9e,algeria,4,SciConsens,PolicySupport|BeliefinAnthrCC|Socialmediashari...,50.0,50.0,53.0,53.0,66.0,...,80.0,35.0,35.0,70.0,26.0,32.0,NaN,-83.0,1,Africa
1,R_3KOO2oUWQNtKzrD,algeria,8,PluralIgnorance,PolicySupport|SocialmediasharingPipedText|Beli...,100.0,100.0,100.0,100.0,1.0,...,80.0,35.0,35.0,70.0,26.0,32.0,NaN,-83.0,1,Africa
2,R_3OdqhjO2quQKaIx,algeria,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,93.0,67.0,100.0,33.0,15.0,...,80.0,35.0,35.0,70.0,26.0,32.0,NaN,-83.0,1,Africa
3,R_2QuK4DRmnWL9FXS,algeria,10,DynamicNorm,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,NaN,3.0,NaN,NaN,0.0,...,80.0,35.0,35.0,70.0,26.0,32.0,NaN,-83.0,1,Africa
4,R_1DYkviJgtp5u6JV,algeria,7,PsychDistance,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,68.0,100.0,100.0,95.0,70.0,...,80.0,35.0,35.0,70.0,26.0,32.0,NaN,-83.0,1,Africa


In [ ]:
# Create dummy variables for 'Continent'
continent_dummies = pd.get_dummies(df['Continent'], prefix='Continent', drop_first=False)

# Concatenate the original DataFrame with the new dummy variables
df = pd.concat([df, continent_dummies], axis=1)
df.head()

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,MinkovKaasa2022,Ind_ms,Continent,Continent_Africa,Continent_Asia,"Continent_Asia, Europe",Continent_Europe,Continent_North America,Continent_Oceania,Continent_South America
0,R_1DNJsF7DGNuIb9e,algeria,4,SciConsens,PolicySupport|BeliefinAnthrCC|Socialmediashari...,50.0,50.0,53.0,53.0,66.0,...,-83.0,1,Africa,True,False,False,False,False,False,False
1,R_3KOO2oUWQNtKzrD,algeria,8,PluralIgnorance,PolicySupport|SocialmediasharingPipedText|Beli...,100.0,100.0,100.0,100.0,1.0,...,-83.0,1,Africa,True,False,False,False,False,False,False
2,R_3OdqhjO2quQKaIx,algeria,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,93.0,67.0,100.0,33.0,15.0,...,-83.0,1,Africa,True,False,False,False,False,False,False
3,R_2QuK4DRmnWL9FXS,algeria,10,DynamicNorm,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,NaN,3.0,NaN,NaN,0.0,...,-83.0,1,Africa,True,False,False,False,False,False,False
4,R_1DYkviJgtp5u6JV,algeria,7,PsychDistance,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,68.0,100.0,100.0,95.0,70.0,...,-83.0,1,Africa,True,False,False,False,False,False,False


In [ ]:
df.rename(columns={'Continent_North America': 'Continent_NorthAmerica'}, inplace=True)
df.rename(columns={'Continent_South America': 'Continent_SouthAmerica'}, inplace=True)

In [ ]:
# The edge case of Russia
# Step 1: Update the 'Continent_Asia' and 'Continent_Europe' columns for Russia
df.loc[df['Continent_Asia, Europe'] == True, 'Continent_Asia'] = True
df.loc[df['Continent_Asia, Europe'] == True, 'Continent_Europe'] = True

# Step 2: Drop the 'Continent_Asia, Europe' column
df = df.drop(columns='Continent_Asia, Europe')
df

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,Notes,MinkovKaasa2022,Ind_ms,Continent,Continent_Africa,Continent_Asia,Continent_Europe,Continent_NorthAmerica,Continent_Oceania,Continent_SouthAmerica
0,R_1DNJsF7DGNuIb9e,algeria,4,SciConsens,PolicySupport|BeliefinAnthrCC|Socialmediashari...,50.0,50.0,53.0,53.0,66.0,...,NaN,-83.0,1,Africa,True,False,False,False,False,False
1,R_3KOO2oUWQNtKzrD,algeria,8,PluralIgnorance,PolicySupport|SocialmediasharingPipedText|Beli...,100.0,100.0,100.0,100.0,1.0,...,NaN,-83.0,1,Africa,True,False,False,False,False,False
2,R_3OdqhjO2quQKaIx,algeria,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,93.0,67.0,100.0,33.0,15.0,...,NaN,-83.0,1,Africa,True,False,False,False,False,False
3,R_2QuK4DRmnWL9FXS,algeria,10,DynamicNorm,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,NaN,3.0,NaN,NaN,0.0,...,NaN,-83.0,1,Africa,True,False,False,False,False,False
4,R_1DYkviJgtp5u6JV,algeria,7,PsychDistance,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,68.0,100.0,100.0,95.0,70.0,...,NaN,-83.0,1,Africa,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59435,R_2TuXye2HP6EO5KZ,vietnam,11,FutureSelfCont,PolicySupport|SocialmediasharingPipedText|Beli...,49.0,100.0,100.0,74.0,70.0,...,NaN,-78.0,1,Asia,False,True,False,False,False,False
59436,R_3R3p8C0gzLKTcCu,vietnam,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,94.0,98.0,98.0,96.0,92.0,...,NaN,-78.0,1,Asia,False,True,False,False,False,False
59437,R_OeT1JV3yC1aJ4EF,vietnam,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,100.0,100.0,100.0,100.0,80.0,...,NaN,-78.0,1,Asia,False,True,False,False,False,False
59438,R_2azVf0KXD3rtly8,vietnam,3,NegativeEmotions,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,79.0,51.0,78.0,51.0,0.0,...,NaN,-78.0,1,Asia,False,True,False,False,False,False


## Interventions’ Impact on Beliefs

In [ ]:
df_belief = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'Continent', 'Continent_NorthAmerica', 'Continent_Europe', 'Continent_Africa','Continent_Asia', 'Continent_SouthAmerica', 'Continent_Oceania', 'condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP", "ide", 'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'Continent', 'Continent_NorthAmerica', 'Continent_Europe', 'Continent_Africa','Continent_Asia', 'Continent_SouthAmerica', 'Continent_Oceania','condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP", "ide"],
    var_name="Item",
    value_name="Belief"
)
df_belief['condName'] = df_belief['condName'].replace(['Control'], 'AControl')

#deal with missing data
df_belief = df_belief.dropna(subset=["Belief"])
%Rpush df_belief

### Table S18. Belief

In [ ]:
%%R
# Add continent fixed effects
df_belief$Continent_Asia <- as.factor(df_belief$Continent_Asia)
df_belief$Continent_NorthAmerica <- as.factor(df_belief$Continent_NorthAmerica)
df_belief$Continent_Europe <- as.factor(df_belief$Continent_Europe)
df_belief$Continent_Africa <- as.factor(df_belief$Continent_Africa)
df_belief$Continent_SouthAmerica <- as.factor(df_belief$Continent_SouthAmerica)
df_belief$Continent_Oceania <- as.factor(df_belief$Continent_Oceania)

M1 <- lmer(Belief ~ condName*Ind_score + Continent_NorthAmerica + Continent_Europe + Continent_Africa + Continent_Asia + Continent_SouthAmerica + Continent_Oceania + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_belief)
print(summary(M1))
summary_M1 <- summary(M1)

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = df_belief, type = "lme4")
print(cohens_d)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
Belief ~ condName * Ind_score + Continent_NorthAmerica + Continent_Europe +  
    Continent_Africa + Continent_Asia + Continent_SouthAmerica +  
    Continent_Oceania + (1 | Country) + (1 | ResponseId) + (1 |      Item)
   Data: df_belief

REML criterion at convergence: 1982335

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2901 -0.2693  0.0885  0.2938  6.0001 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 502.0761 22.4071 
 Country    (Intercept)  26.0474  5.1037 
 Item       (Intercept)   0.2813  0.5304 
 Residual               147.4645 12.1435 
Number of obs: 233116, groups:  ResponseId, 58370; Country, 62; Item, 4

Fixed effects:
                                     Estimate Std. Error         df t value
(Intercept)                         9.106e+01  5.776e+00  5.374e+01  15.766
condNameBindingMoral               -5.171e-01  1.122e+00


Correlation matrix not shown by default, as p = 30 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it

In addition: Warning messages:
1: In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00641932 (tol = 0.002, component 1)
2: In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00641932 (tol = 0.002, component 1)


## Interventions’ Impact on Policy Support

In [ ]:
df_policy = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'Continent', 'Continent_NorthAmerica', 'Continent_Europe', 'Continent_Africa','Continent_Asia', 'Continent_SouthAmerica', 'Continent_Oceania','condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP", "ide",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'Continent', 'Continent_NorthAmerica', 'Continent_Europe', 'Continent_Africa','Continent_Asia', 'Continent_SouthAmerica', 'Continent_Oceania','condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP", "ide"],
    var_name="Item",
    value_name="Policy"
)
df_policy['condName'] = df_policy['condName'].replace(['Control'], 'AControl')

#deal with missing data
df_policy = df_policy.dropna(subset=["Policy"])

%Rpush df_policy

### Table S19. Policy Support

In [ ]:
%%R
# Add continent fixed effects
df_policy$Continent_Asia <- as.factor(df_policy$Continent_Asia)
df_policy$Continent_NorthAmerica <- as.factor(df_policy$Continent_NorthAmerica)
df_policy$Continent_Europe <- as.factor(df_policy$Continent_Europe)
df_policy$Continent_Africa <- as.factor(df_policy$Continent_Africa)
df_policy$Continent_SouthAmerica <- as.factor(df_policy$Continent_SouthAmerica)
df_policy$Continent_Oceania <- as.factor(df_policy$Continent_Oceania)

M1 <- lmer(Policy ~ condName*Ind_score + Continent_NorthAmerica + Continent_Europe + Continent_Africa + Continent_Asia + Continent_SouthAmerica + Continent_Oceania + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_policy)
print(summary(M1))
summary_M1 <- summary(M1)

#rsq <- r2(M1) #marginal and conditional R² values
#print(rsq)

# Calculate Cohen's d
#cohens_d <- lme.dscore(M1, data = df_belief, type = "lme4")
#print(cohens_d)


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
Policy ~ condName * Ind_score + Continent_NorthAmerica + Continent_Europe +  
    Continent_Africa + Continent_Asia + Continent_SouthAmerica +  
    Continent_Oceania + (1 | Country) + (1 | ResponseId) + (1 |      Item)
   Data: df_policy

REML criterion at convergence: 4615918

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.7686 -0.5203  0.0709  0.6338  4.6549 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 307.22   17.528  
 Country    (Intercept)  16.27    4.033  
 Item       (Intercept) 140.60   11.858  
 Residual               429.63   20.728  
Number of obs: 505706, groups:  ResponseId, 57970; Country, 62; Item, 9

Fixed effects:
                                     Estimate Std. Error         df t value
(Intercept)                         8.088e+01  6.039e+00  3.398e+01  13.393
condNameBindingMoral                5.718e-01  9.172e-01


Correlation matrix not shown by default, as p = 30 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it



## Interventions’ Impact on Social Media Sharing

In [ ]:
#deal with missing data
df_share = df.dropna(subset=["SHAREcc"])

df_share['condName'] = df_share['condName'].replace(['Control'], 'AControl')

%Rpush df_share

### Table S20. Sharing

In [ ]:
%%R
# Add continent fixed effects
df_share$Continent_Asia <- as.factor(df_share$Continent_Asia)
df_share$Continent_NorthAmerica <- as.factor(df_share$Continent_NorthAmerica)
df_share$Continent_Europe <- as.factor(df_share$Continent_Europe)
df_share$Continent_Africa <- as.factor(df_share$Continent_Africa)
df_share$Continent_SouthAmerica <- as.factor(df_share$Continent_SouthAmerica)
df_share$Continent_Oceania <- as.factor(df_share$Continent_Oceania)

M1 <- lmer(Policy ~ condName*Ind_score + Continent_NorthAmerica + Continent_Europe + Continent_Africa + Continent_Asia + Continent_SouthAmerica + Continent_Oceania + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_policy)
df_share$Ind_score <- scale(df_share$Ind_score)
#this code rescales the predictor variable to address large eigenvalues

M1 <- glmer(SHAREcc ~ condName*Ind_score + Continent_NorthAmerica + Continent_Europe + Continent_Africa + Continent_Asia + Continent_SouthAmerica + Continent_Oceania + (1 | Country), 
            family = binomial, 
            data = df_share, 
            control = glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 100000)))

print(summary(M1))
summary_M1 <- summary(M1)

#rsq <- r2(M1) #marginal and conditional R² values
#print(rsq)

# Calculate Cohen's d
cohens_d <- lme.dscore(M1, data = df_share, type = "lme4")
print(cohens_d)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
SHAREcc ~ condName * Ind_score + Continent_NorthAmerica + Continent_Europe +  
    Continent_Africa + Continent_Asia + Continent_SouthAmerica +  
    Continent_Oceania + (1 | Country)
   Data: df_share
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 1e+05))

     AIC      BIC   logLik deviance df.resid 
 51783.3  52052.6 -25860.7  51721.3    43699 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2878 -0.7428  0.3778  0.7823  2.5018 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.5489   0.7409  
Number of obs: 43730, groups:  Country, 62

Fixed effects:
                                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)                         0.959535   0.623386   1.539 0.123748    
condNameBindingMoral                0.199351   0.050658   3.935 8.31e-05 ***
condNameC


Correlation matrix not shown by default, as p = 30 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it

In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00315654 (tol = 0.002, component 1)


## Interventions’ Impact on Tree Planting Behaviors

In [ ]:
#deal with missing data
df_WEPT = df.dropna(subset=["WEPTcc"])

df_WEPT['condName'] = df_WEPT['condName'].replace(['Control'], 'AControl')

%Rpush df_WEPT

### Table S21. Tree Planting

In [ ]:
%%R
# Add continent fixed effects
df_WEPT$Continent_Asia <- as.factor(df_WEPT$Continent_Asia)
df_WEPT$Continent_NorthAmerica <- as.factor(df_WEPT$Continent_NorthAmerica)
df_WEPT$Continent_Europe <- as.factor(df_WEPT$Continent_Europe)
df_WEPT$Continent_Africa <- as.factor(df_WEPT$Continent_Africa)
df_WEPT$Continent_SouthAmerica <- as.factor(df_WEPT$Continent_SouthAmerica)
df_WEPT$Continent_Oceania <- as.factor(df_WEPT$Continent_Oceania)

df_WEPT$Ind_score <- scale(df_WEPT$Ind_score)
#this code rescales the predictor variable to address large eigenvalues

M <- clmm(as.factor(WEPTcc) ~ condName*Ind_score + Continent_NorthAmerica + Continent_Europe + Continent_Africa + Continent_Asia + Continent_SouthAmerica + Continent_Oceania + (1|Country), data = df_WEPT, threshold = "equidistant")
#summary(M)
print(summary(M))
summary_M <- summary(M)

# Extract fixed effects coefficients directly
model_coef <- coef(M)

# Calculate odds ratios for fixed effects
odds_ratios <- exp(model_coef)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ condName * Ind_score + Continent_NorthAmerica +  
    Continent_Europe + Continent_Africa + Continent_Asia + Continent_SouthAmerica +  
    Continent_Oceania + (1 | Country)
data:    df_WEPT

 link  threshold   nobs  logLik    AIC       niter       max.grad cond.H 
 logit equidistant 58979 -96391.81 192847.61 8039(32161) 5.25e+00 9.8e+05

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2098   0.458   
Number of groups:  Country 62 

Coefficients:
                                    Estimate Std. Error z value Pr(>|z|)    
condNameBindingMoral                0.038997   0.038060   1.025  0.30555    
condNameCollectAction              -0.113442   0.037561  -3.020  0.00253 ** 
condNameDynamicNorm                 0.013674   0.037732   0.362  0.71705    
condNameFutureSelfCont             -0.116855   0.039595  -2.951  0.00316 ** 
condNameLetterFutureGen            -0.

In addition: Warning message:
(-1) Model failed to converge with max|grad| = 0.000876746 (tol = 1e-06) 
In addition: step factor reduced below minimum 


# Robustness checks with alternate individualism measures

## Belief

In [33]:
dfc_belief = pd.melt(
    dfc.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide',  
                'Belief1', 'Belief2', 'Belief3', 'Belief4', "RelMobility", "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"]],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide', 
             "RelMobility",  "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"],
    var_name="Item",
    value_name="Belief"
)

dfc_belief = dfc_belief.dropna(subset=["Belief"]) #deal with missing data for key outcome variable
%Rpush dfc_belief

### Table S32. Minkov & Kaasa 2022 individualism measure

In [35]:
%%R

M1 <- lmer(Belief ~ MinkovKaasa2022 + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_1_minkovkaasa.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ MinkovKaasa2022 + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 158015.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.1726 -0.2852  0.0853  0.3183  5.8133 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 530.7488 23.0380 
 Country    (Intercept)  25.3280  5.0327 
 Item       (Intercept)   0.3332  0.5772 
 Residual               152.4449 12.3469 
Number of obs: 18491, groups:  ResponseId, 4628; Country, 54; Item, 4

Fixed effects:
                 Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)     79.818253   0.855095 50.606437  93.344  < 2e-16 ***
MinkovKaasa2022 -0.033345   0.008385 52.531793  -3.977 0.000215 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
MinkvKs2022 0.006 


### Table S33. GLOBE variable (in-group collectivism)

In [39]:
%%R

M1 <- lmer(Belief ~ GLOBECol2SocPracInGroup + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_1_globeingroup.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ GLOBECol2SocPracInGroup + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 120838.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.1742 -0.3029  0.0884  0.3258  5.9804 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 566.5603 23.8025 
 Country    (Intercept)  27.3159  5.2265 
 Item       (Intercept)   0.2271  0.4765 
 Residual               145.8195 12.0756 
Number of obs: 14173, groups:  ResponseId, 3547; Country, 37; Item, 4

Fixed effects:
                        Estimate Std. Error     df t value Pr(>|t|)    
(Intercept)               60.973      6.043 34.973  10.091 6.75e-12 ***
GLOBECol2SocPracInGroup    3.663      1.220 36.704   3.003  0.00479 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
GLOBEC2SP

### Table S34. GLOBE variable (institutional collectivism)

In [40]:
%%R

M1 <- lmer(Belief ~ GLOBECol1SocValInstit + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_1_globeinst.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ GLOBECol1SocValInstit + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 120833.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.1770 -0.3029  0.0867  0.3263  5.9803 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 566.836  23.8083 
 Country    (Intercept)  22.331   4.7256 
 Item       (Intercept)   0.227   0.4765 
 Residual               145.820  12.0756 
Number of obs: 14173, groups:  ResponseId, 3547; Country, 37; Item, 4

Fixed effects:
                      Estimate Std. Error     df t value Pr(>|t|)    
(Intercept)             46.429      8.624 32.736   5.384 6.09e-06 ***
GLOBECol1SocValInstit    7.066      1.872 33.696   3.775  0.00062 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
GLOBECl1SVI -0.99

In [38]:
# create correlation matrix

# Subset the dataframe to relevant columns
subset_df = dfc[["Ind_score", "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit"]]

# Compute the correlation matrix
cor_matrix = subset_df.corr()

print(cor_matrix)

                         Ind_score  GLOBECol2SocPracInGroup  \
Ind_score                 1.000000                -0.741529   
GLOBECol2SocPracInGroup  -0.741529                 1.000000   
GLOBECol1SocValInstit    -0.422430                 0.477257   

                         GLOBECol1SocValInstit  
Ind_score                            -0.422430  
GLOBECol2SocPracInGroup               0.477257  
GLOBECol1SocValInstit                 1.000000  


## Policy

In [41]:
dfc_policy = pd.melt(
    dfc.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide', 
                'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9', "RelMobility",
                "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"]],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Income", "Edu", "Age", "Gender", "GDP", 'ide',  "RelMobility",
             "GLOBECol2SocPracInGroup", "GLOBECol1SocValInstit", "Parochialism", "MinkovKaasa2022"],
    var_name="Item",
    value_name="Policy"
)

dfc_policy = dfc_policy.dropna(subset=["Policy"]) #deal with missing data
%Rpush dfc_policy

### Table S35. Minkov & Kaasa 2022 individualism measure

In [42]:
%%R

M1 <- lmer(Policy ~ MinkovKaasa2022 + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_2_minkovkaasa.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ MinkovKaasa2022 + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 366890.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5799 -0.5264  0.0652  0.6318  4.1136 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 315.13   17.752  
 Country    (Intercept)  14.18    3.766  
 Item       (Intercept) 150.76   12.278  
 Residual               431.15   20.764  
Number of obs: 40159, groups:  ResponseId, 4600; Country, 54; Item, 9

Fixed effects:
                 Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)     69.754393   4.138587  8.352358  16.855 9.64e-08 ***
MinkovKaasa2022 -0.022441   0.006392 53.813650  -3.511 0.000912 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
MinkvKs2022 0.000 


### Table S36. GLOBE variable (in-group collectivism)

In [43]:
%%R

M1 <- lmer(Policy ~ GLOBECol2SocPracInGroup + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_2_globeingroup.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ GLOBECol2SocPracInGroup + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 280337.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4730 -0.5422  0.0524  0.6324  4.3269 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 340.59   18.455  
 Country    (Intercept)  13.49    3.673  
 Item       (Intercept) 157.78   12.561  
 Residual               400.98   20.025  
Number of obs: 30881, groups:  ResponseId, 3529; Country, 37; Item, 9

Fixed effects:
                        Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)              56.2455     6.0643 27.8237   9.275 5.24e-10 ***
GLOBECol2SocPracInGroup   2.6268     0.8887 38.8473   2.956  0.00528 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
GLOBEC

### Table S37. GLOBE variable (institutional collectivism)

In [44]:
%%R

M1 <- lmer(Policy ~ GLOBECol1SocValInstit + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_2_globeinst.csv") #export coefficients to csv

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ GLOBECol1SocValInstit + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 280330

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4800 -0.5420  0.0524  0.6324  4.3266 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 340.835  18.462  
 Country    (Intercept)   9.217   3.036  
 Item       (Intercept) 157.777  12.561  
 Residual               400.985  20.025  
Number of obs: 30881, groups:  ResponseId, 3529; Country, 37; Item, 9

Fixed effects:
                      Estimate Std. Error     df t value Pr(>|t|)    
(Intercept)             43.300      7.261 36.237   5.963  7.6e-07 ***
GLOBECol1SocValInstit    5.612      1.292 33.700   4.345 0.000121 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
GLOBECl1SVI -0.812


## Sharing

In [45]:
dfc_share = dfc.dropna(subset=["SHAREcc"]) #deal with missing data
%Rpush dfc_share

### Table S38. Minkov & Kaasa 2022 individualism measure

In [46]:
%%R

M1 <- glmer(SHAREcc ~ MinkovKaasa2022 + (1 | Country), family = binomial, data = dfc_share)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_3_minkovkaasa.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ MinkovKaasa2022 + (1 | Country)
   Data: dfc_share

     AIC      BIC   logLik deviance df.resid 
  4419.9   4438.4  -2206.9   4413.9     3568 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6462 -0.7524 -0.4096  0.8668  2.6671 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.8171   0.9039  
Number of obs: 3571, groups:  Country, 54

Fixed effects:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -0.064786   0.132168  -0.490    0.624    
MinkovKaasa2022 -0.006174   0.001384  -4.462 8.14e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
MinkvKs2022 0.043 
optimizer (Nelder_Mead) convergence code: 0 (OK)
Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?



In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?


### Table S39. GLOBE variable (in-group collectivism)

In [47]:
%%R

M1 <- glmer(SHAREcc ~ GLOBECol2SocPracInGroup + (1 | Country), family = binomial, data = dfc_share)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_3_globeingroup.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ GLOBECol2SocPracInGroup + (1 | Country)
   Data: dfc_share

     AIC      BIC   logLik deviance df.resid 
  3392.3   3410.0  -1693.1   3386.3     2677 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.1038 -0.9190 -0.4160  0.9357  2.8159 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.7213   0.8493  
Number of obs: 2680, groups:  Country, 37

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -3.2172     0.9149  -3.517 0.000437 ***
GLOBECol2SocPracInGroup   0.6474     0.1833   3.531 0.000414 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
GLOBEC2SPIG -0.986


### Table S40. GLOBE variable (institutional collectivism)

In [48]:
%%R

M1 <- glmer(SHAREcc ~ GLOBECol1SocValInstit + (1 | Country), family = binomial, data = dfc_share)
print(summary(M1))

#write.csv(coef(summary(M1)), "1_3_globeinst.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ GLOBECol1SocValInstit + (1 | Country)
   Data: dfc_share

     AIC      BIC   logLik deviance df.resid 
  3395.9   3413.6  -1695.0   3389.9     2677 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.1654 -0.9344 -0.4046  0.9256  2.7177 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.8077   0.8987  
Number of obs: 2680, groups:  Country, 37

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)   
(Intercept)            -4.1713     1.4776  -2.823  0.00476 **
GLOBECol1SocValInstit   0.9004     0.3193   2.820  0.00480 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr)
GLOBECl1SVI -0.994


## Tree Planting

In [49]:
dfc_WEPT = dfc.dropna(subset=["WEPTcc"]) # deal with missing data
%Rpush dfc_WEPT

### Table S41. Minkov & Kaasa 2022 individualism measure

In [50]:
%%R

M1 <- clmm(as.factor(WEPTcc) ~ MinkovKaasa2022 + (1 | Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
print(summary(M1))

# Extract fixed effects coefficients directly
model_coef <- coef(M1)

# Calculate odds ratios for fixed effects
odds_ratios <- exp(model_coef)

#write.csv(coef(summary(M1)), "1_4_minkovkaasa.csv") #export coefficients to csv

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ MinkovKaasa2022 + (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 4678 -7419.54 14847.08 235(717) 4.85e-03 3.8e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1727   0.4156  
Number of groups:  Country 54 

Coefficients:
                  Estimate Std. Error z value Pr(>|z|)
MinkovKaasa2022 -0.0005900  0.0006834  -0.863    0.388

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -1.345394   0.068582  -19.62
spacing      0.184788   0.004483   41.22
(407 observations deleted due to missingness)


### Table S42. GLOBE variable (in-group collectivism)

In [51]:
%%R

M1 <- clmm(as.factor(WEPTcc) ~ GLOBECol2SocPracInGroup + (1 | Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
print(summary(M1))

# Extract fixed effects coefficients directly
model_coef <- coef(M1)

# Calculate odds ratios for fixed effects
odds_ratios <- exp(model_coef)

#write.csv(coef(summary(M1)), "1_4_globeingroup.csv") #export coefficients to csv

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ GLOBECol2SocPracInGroup + (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 3551 -5678.71 11365.42 162(491) 1.23e-03 1.1e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2101   0.4584  
Number of groups:  Country 37 

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)
GLOBECol2SocPracInGroup  0.01696    0.10375   0.163     0.87

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -1.188595   0.515114  -2.307
spacing      0.183868   0.005105  36.014
(1534 observations deleted due to missingness)


### Table S43. GLOBE variable (institutional collectivism)

In [52]:
%%R

M1 <- clmm(as.factor(WEPTcc) ~ GLOBECol1SocValInstit + (1 | Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
print(summary(M1))

# Extract fixed effects coefficients directly
model_coef <- coef(M1)

# Calculate odds ratios for fixed effects
odds_ratios <- exp(model_coef)

#write.csv(coef(summary(M1)), "1_4_globeinst.csv") #export coefficients to csv

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ GLOBECol1SocValInstit + (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 3551 -5678.57 11365.13 123(374) 1.36e-02 2.7e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.2072   0.4552  
Number of groups:  Country 37 

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)
GLOBECol1SocValInstit  0.09579    0.17194   0.557    0.577

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -0.831729   0.793998  -1.048
spacing      0.183857   0.005105  36.012
(1534 observations deleted due to missingness)
